In [6]:
import numpy as np
import pprint
import operator


In [1]:
from typing import Generic, TypeVar, Dict, List, Optional, Tuple
from abc import ABC, abstractmethod

V = TypeVar('V') # variable type
D = TypeVar('D') # domain type

# Base class for all constraints
class Constraint(Generic[V, D], ABC):
    # The variables that the constraint is between
    def __init__(self, variables: List[V]) -> None:
        self.variables = variables

    # Must be overridden by subclasses
    @abstractmethod
    def satisfied(self, assignment: Dict[V, D]) -> bool:
        ...


# A constraint satisfaction problem consists of variables of type V
# that have ranges of values known as domains of type D and constraints
# that determine whether a particular variable's domain selection is valid

class CSP(Generic[V, D]):
    def __init__(self, variables: List[V], domains: Dict[V, List[D]]) -> None:
        self.variables:   List[V]                         = variables # variables to be constrained
        self.domains:     Dict[V, List[D]]                = domains   # domain of each variable
        self.constraints: Dict[V, List[Constraint[V, D]]] = {}
            
        for variable in self.variables:
            self.constraints[variable] = []
            if variable not in self.domains:
                raise LookupError("Every variable should have a domain assigned to it.")

    def add_constraint(self, constraint: Constraint[V, D]) -> None:
        for variable in constraint.variables:
            if variable not in self.variables:
                raise LookupError("Variable in constraint not in CSP")
            else:
                self.constraints[variable].append(constraint)

    # Check if the value assignment is consistent by checking all constraints
    # for the given variable against it
    def consistent(self, variable: V, assignment: Dict[V, D]) -> bool:
        for constraint in self.constraints[variable]:
            if not constraint.satisfied(assignment):
                return False
        return True

    def backtracking_search(self, assignment: Dict[V, D] = {}) -> Optional[Dict[V, D]]:
        # assignment is complete if every variable is assigned (our base case)
        if len(assignment) == len(self.variables):
            return assignment

        # get all variables in the CSP but not in the assignment
        unassigned: List[V] = [v for v in self.variables if v not in assignment]

        # get the every possible domain value of the first unassigned variable
        first: V = unassigned[0]
        for value in self.domains[first]:
            local_assignment = assignment.copy()
            local_assignment[first] = value
            # if we're still consistent, we recurse (continue)
            if self.consistent(first, local_assignment):
                result: Optional[Dict[V, D]] = self.backtracking_search(local_assignment)
                # if we didn't find the result, we will end up backtracking
                if result is not None:
                    return result
        return None

In [20]:
class FirstConstraint(Constraint[Tuple, int]):
    
    def __init__(self, variables: List[Tuple], problem_size: int) -> None:
        super().__init__(variables)
        self.ps        = problem_size
        self.matrix    = np.empty((problem_size, problem_size))
        self.matrix[:] = np.nan

    def satisfied(self, assignment: Dict[Tuple, int]) -> bool:
        
        def U(cordinate: Tuple): # Up
            return tuple(map(operator.add, cordinate, (-1,  0)))

        def D(cordinate: Tuple): # Down
            return tuple(map(operator.add, cordinate, (+1,  0)))

        def L(cordinate: Tuple): # Left
            return tuple(map(operator.add, cordinate, ( 0, -1)))

        def R(cordinate: Tuple): # Right
            return tuple(map(operator.add, cordinate, ( 0, +1)))
        
        matrix = self.matrix
        matrix[:] = np.nan
        for var in assignment:
            matrix[var] = assignment[var]
        
        
        for var in assignment:
            if var in [(0, 0), (self.ps - 1, self.ps - 1), (self.ps - 1, 0), (0, self.ps - 1)]:
                continue

            elif var[0] in [0, self.ps - 1]:
                if matrix[L(var)] == matrix[var] == matrix[R(var)]:
                    return False
                
            elif var[1] in [0, self.ps - 1]:
                if matrix[U(var)] == matrix[var] == matrix[D(var)]:
                    return False

            else:
                if matrix[L(var)] == matrix[var] == matrix[R(var)]:
                    return False
                if matrix[U(var)] == matrix[var] == matrix[D(var)]:
                    return False

        return True
    
    
class SecondConstraint(Constraint[Tuple, int]):
    
    def __init__(self, variables: List[Tuple], problem_size: int) -> None:
        super().__init__(variables)
        self.ps        = problem_size
        self.matrix    = np.empty((self.ps, self.ps))
        self.matrix[:] = np.nan

    def satisfied(self, assignment: Dict[Tuple, int]) -> bool:
        
        matrix = self.matrix
        matrix[:] = np.nan
        for var in assignment:
            matrix[var] = assignment[var]
        
        for i in range(0, self.ps):
            if not np.isnan(matrix[i,:]).any():
                if np.sum(matrix[i,:]) != self.ps / 2.0:
                    return False
                
        for i in range(0, self.ps):
            if not np.isnan(matrix[:,i]).any():
                if np.sum(matrix[:,i]) != self.ps / 2.0:
                    return False
                
        return True
    

class ThirdConstraint(Constraint[Tuple, int]):
    
    def __init__(self, variables: List[Tuple], problem_size: int) -> None:
        super().__init__(variables)
        self.ps        = problem_size
        self.matrix    = np.empty((self.ps, self.ps))
        self.matrix[:] = np.nan

    def satisfied(self, assignment: Dict[Tuple, int]) -> bool:
        
        matrix = self.matrix
        matrix[:] = np.nan
        for var in assignment:
            matrix[var] = assignment[var]
        
        for i in range(self.ps):  # generate pairs
            for j in range(i + 1, self.ps):
                if not np.isnan(matrix[i]).any() and np.isnan(matrix[j]).any():
                    if np.array_equal(matrix[i], matrix[j]):  # compare rows
                        return False
                

        for i in range(self.ps):  # generate pairs
            for j in range(i + 1, self.ps): 
                if not np.isnan(matrix[:, i]).any() and np.isnan(matrix[:, j]).any():
                    if np.array_equal(matrix[:,i], matrix[:, j]):  # compare columns
                        return False

        return True


In [22]:

    
s = 10
r = range(0, s)

domains:   Dict[Tuple, List[int]] = {(i, j):[0, 1] for i in r for j in r}
variables: List[Tuple]            = domains.keys()
csp: CSP[Tuple, int] = CSP(variables, domains)
    
csp.add_constraint(FirstConstraint(variables, s))
csp.add_constraint(SecondConstraint(variables, s))
csp.add_constraint(ThirdConstraint(variables, s))

solution: Optional[Dict[int, int]] = csp.backtracking_search()
if solution is None:
    print("No solution found!")
else:
    matrix    = np.empty((s, s))
    matrix[:] = np.nan

    for var in solution:
        matrix[var] = solution[var]
    print(matrix)
    

[[0. 0. 1. 0. 0. 1. 1. 0. 1. 1.]
 [0. 0. 1. 0. 0. 1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 1. 1. 0. 1. 1.]
 [0. 0. 1. 0. 0. 1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1. 0. 0. 1. 0. 0.]
 [1. 1. 0. 1. 1. 0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0. 1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 1. 0. 0. 1. 0. 0.]
 [1. 1. 0. 1. 1. 0. 0. 1. 0. 0.]]


In [11]:
import random
v = {}
for i in [random.randrange(0,10) for m in range(0,20)]:
    for j in [random.randrange(0, 10) for m in range(0, 20)]:
        v[(i, j)] = random.randrange(0, 2)
        
# pprint.pprint(v)
    
a = np.empty((10,10))
a[:] = np.nan

for cor in v:
    a[cor] = v[cor]
    
for i in range(0, 10):
    if not np.isnan(a[i,:]).any():
        print(np.sum(a[i,:]))
    
print('\n\n\n')  
print(a[np.isnan(a[i,:]).any(), :])
print('\n\n\n')  
print(a)
print(len(a))
print('\n\n\n')
print(a[:, 0]!=np.nan)
print('\n\n\n')
print(a[a[:,0] == np.nan, :])

7.0
4.0
5.0
2.0
5.0
2.0




[[[nan nan nan nan nan nan nan nan nan nan]
  [ 0.  1.  0.  1.  1.  1.  1.  1.  1.  0.]
  [ 1.  1.  0.  0.  1.  0.  0.  1.  0.  0.]
  [ 1.  1.  1.  0.  0. nan  0.  1.  1.  1.]
  [ 1.  0.  1.  0.  0.  1.  0. nan  1.  1.]
  [ 0.  0.  1.  0.  0.  1.  1.  0.  1.  1.]
  [ 0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
  [ 0.  1.  0.  1.  0.  1.  0.  0.  1.  1.]
  [ 0.  0.  0.  0.  0.  0.  1.  0.  1.  0.]
  [ 0.  0.  1.  0.  0. nan  0.  0.  0.  0.]]]




[[nan nan nan nan nan nan nan nan nan nan]
 [ 0.  1.  0.  1.  1.  1.  1.  1.  1.  0.]
 [ 1.  1.  0.  0.  1.  0.  0.  1.  0.  0.]
 [ 1.  1.  1.  0.  0. nan  0.  1.  1.  1.]
 [ 1.  0.  1.  0.  0.  1.  0. nan  1.  1.]
 [ 0.  0.  1.  0.  0.  1.  1.  0.  1.  1.]
 [ 0.  1.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  1.  0.  1.  0.  1.  0.  0.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  1.  0.]
 [ 0.  0.  1.  0.  0. nan  0.  0.  0.  0.]]
10




[ True  True  True  True  True  True  True  True  True  True]




[]


In [19]:
A=np.array([[0, 1, 0, 0, 0, 1],
            [0, 0, 0, 1, 0, 1],
            [0, 1, 0, 0, 0, 1],
            [1, 0, 1, 0, 1, 1],
            [1, 1, 1, 0, 0, 0],
            [0, 1, 0, 1, 0, 1]])

A = np.ones((10, 10))


for i in range(len(A)):  # generate pairs
    for j in range(i + 1, len(A)): 
        if np.array_equal(A[i], A[j]):  # compare rows
            print(i, j)
        else:
            pass
        
print('-------')
for i in range(len(A)):  # generate pairs
    for j in range(i + 1, len(A)): 
        if np.array_equal(A[:,i], A[:,j]):  # compare rows
            print(i, j)
        else:
            pass

0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
2 3
2 4
2 5
2 6
2 7
2 8
2 9
3 4
3 5
3 6
3 7
3 8
3 9
4 5
4 6
4 7
4 8
4 9
5 6
5 7
5 8
5 9
6 7
6 8
6 9
7 8
7 9
8 9
-------
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
2 3
2 4
2 5
2 6
2 7
2 8
2 9
3 4
3 5
3 6
3 7
3 8
3 9
4 5
4 6
4 7
4 8
4 9
5 6
5 7
5 8
5 9
6 7
6 8
6 9
7 8
7 9
8 9


In [ ]:
x = np.nan
if x == 0 and ~np.isnan(x):
    print("return")

In [16]:
np.ones((3,3))

array([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]])

In [ ]:
np.nan is np.nan

In [ ]:
r = range(0, 10)
{(i, j):[1, 2] for i in r for j in r}